# Read SUMMA output

In [ ]:
# netcdf/numpy/xray/stats
import numpy as np
from datetime import datetime, timedelta
import pandas as pd
import xarray as xr
from scipy.stats.stats import pearsonr

# OS interaction
import sys, pickle, os

# import plotting
import seaborn as sns
import matplotlib
from matplotlib.pyplot import subplots
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.basemap import Basemap
import kgraph

# Offline Turbulence Package
import turbpy

# Customize
sns.set_style("whitegrid")
sns.set_context('talk')
%matplotlib inline

In [ ]:
# --------------------------------------------------------------------------------------------------------------------
# Directory Lists
# Unix
if 'linux' in sys.platform:
    dirPre = '/home/lapok/gdrive/'
# Mac
elif 'darwin' in sys.platform:
    dirPre = '/Users/karllapo/gdrive/'

dirProj = dirPre + 'SnowHydrology/proj/ModTsfc/'
dirPrint = dirProj + 'Graphics'
dirData = dirProj + 'data'

## Functions to quickly evaluate all model output

In [ ]:
def evalSWE(ds, obs):
    color = kgraph.SRON(ds.expID.size)
    # SWE
    fig, ax = plt.subplots(1, 1, figsize=(15, 10))
    for n, ex in enumerate(ds.expID):
        ax.plot(ds.time, ds.sel(expID=ex).scalarSWE, label=ex.values, color=color[n])
    ax.set_ylabel('SWE (mm)')
    plt.legend(loc='best')
    return fig
    
def evalSurfTemp(dsIn, obs, axmin=-30, vmax=20):
    # Colors
    cmin = 0
    cmax = vmax
    ncol = 7
    dcol = (cmax - cmin) / (ncol - 1)
    cmap = sns.cubehelix_palette(ncol, start=2.6, rot=.075, light=0.8, as_cmap=True)
    cmap = kgraph.cmap_discretize(cmap, ncol)
    
    # Tsurf scatter plot
    numExp = dsIn.expID.values.size
    if numExp <= 4:
        fig, axes = plt.subplots(2, 2, figsize=(5, 5), sharex=True, sharey=True)
        axes = axes.flatten()
        lastRow = 2
    elif (numExp > 4) and (numExp <= 9):
        fig, axes = plt.subplots(3, 3, figsize=(8, 8), sharex=True, sharey=True)
        axes = axes.flatten()
        lastRow = 6
    elif numExp > 9:
        fig, axes = plt.subplots(3, 4, figsize=(10.5, 8), sharex=True, sharey=True)
        axes = axes.flatten()
        lastRow = 8
    
    for nEx, ex in enumerate(dsIn.expID):
        ax = axes[nEx]
        
        ds = dsIn.sel(expID=ex)
        ds['Tsrf'] = obs.Tsrf
        ds['scalarSurfaceTemp'] = ds['scalarSurfaceTemp'] - 273
        
        Y = ds.scalarSurfaceTemp
        X = obs.Tsrf
        H, xedges, yedges = np.histogram2d(X, Y, bins=(np.arange(axmin, 0, 1.5),
                                                       np.arange(axmin, 0, 1.5)))
        H = np.ma.masked_array(H, H < 1)
        X, Y = np.meshgrid(xedges, yedges)
        im = ax.pcolormesh(X.T, Y.T, H, cmap=cmap, vmin=cmin, vmax=cmax)
        
        ax.plot([axmin, 0], [axmin, 0], 'k--')
        if nEx >= lastRow:
            ax.set_xlabel('Obs Ts')
        ax.set_ylabel('Model Ts')
        ax.set_xlim(axmin, 0)
        ax.set_ylim(axmin, 0)
        ax.grid()
        ax.set_title(ex.values)
        
        binTs = np.arange(axmin, 0, 1)
        groupModelTs = proxyGroupby(ds, 'Tsrf', binTs, 'scalarSurfaceTemp')
        ax.plot(binTs, groupModelTs, 'r')
    fig.autofmt_xdate()
    return fig

# Group a variable according to a proxy variable (e.g., group air temperature according to the stability)
def proxyGroupby(ds, binVar, binVals, digitVar):
    proxyGroups = np.digitize(ds[binVar].values, binVals, right=False)
    groupVar = np.zeros(np.size(binVals))
    for bnum, bV in enumerate(binVals):
        groupVar[bnum] = np.nanmean(ds[digitVar][proxyGroups == bnum])
    return groupVar

def evalDiurnalTs(ds, obs):
    color = kgraph.SRON(ds.expID.size)
    # Tsurf scatter plot
    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    hours = np.arange(0, 24)
    for n, ex in enumerate(ds.expID):
        temp = ds.sel(expID=ex)
        ax.plot(hours, temp.scalarSurfaceTemp.groupby('time.hour').mean(dim='time') - 273.15,
                label=ex.values, color=color[n])
    ax.plot(hours, obs.snowTs.groupby('time.hour').mean(dim='time'), 'k', linewidth=3, label='Obs.')
    ax.plot([0, 23], [0, 0], color='.5')
    ax.set_xlim(0, 23)
    ax.set_xlabel('Hour of Day')
    ax.set_ylabel('T ($^{\circ}$)')
    plt.legend(loc='best')
    return fig

## Evaluation data

In [ ]:
os.chdir(dirData)
CDP = xr.open_dataset('CDP.ModTsfc.nc')
CDP = CDP.loc[dict(time = slice(datetime(2002, 10, 1), datetime(2010, 9, 30, 22, 0, 0)))]
CDP_daily = xr.open_dataset('CDP.ModTsfc.Daily.nc')
CDP_daily = CDP_daily.loc[dict(time = slice(datetime(2002, 10, 1), datetime(2010, 9, 30, 22, 0, 0)))]

## Model output

In [ ]:
os.chdir(dirData + '/summaResults')
CDP_internal = xr.open_dataset('CDP.summaResults.internal.nc')
CDP_forcing = xr.open_dataset('CDP.summaResults.forcing.nc')

## Experiment names

In [ ]:
siteNames = ['CDP']
siteYears = {'CDP': np.arange(2002, 2010)}
d1 = datetime(2002, 10, 1)
d2 = datetime(2010, 9, 30, 23, 0, 0)

# ----------
# Experiment names - Structure Uncertainty
experiments_internal = ['layers', 'thermalCond', 'stability']

# Layer
expNames = {'layers': ['layerThin', 'layerThick', 'jrdn1991', 'CLM_2010']}

# Thermal conductivity
arrayNames = ['tyen1965', 'melr1977', 'jrdn1991', 'smnv2000']
arrayAppend = ['Mid', 'Low', 'High']
expNames['thermalCond'] = [name + '_' + append for name in arrayNames for append in arrayAppend]

# Stability
arrayAppend = ['Low', 'Mid', 'High']
arrayNames = ['standard', 'louisinv', 'mahrtexp']
expNames['stability'] = [name + '_' + append for name in arrayNames for append in arrayAppend]

# ----------
# Experiment names - Met Uncertainty
experiments_forcing = ['longwaveBias', 'shortwaveBias', 'specHumidBias',
                       'airTempBias', 'windBias']
metName = {'longwaveBias': 'LWRadAtm',
           'shortwaveBias': 'SWRadAtm',
           'specHumidBias': 'spechum',
           'airTempBias': 'airtemp',
           'windBias': 'windspd'}

# Experiment IDs and values
expNames['longwaveBias'] = ['lowLW_L','lowLW_S', 'highLW_S', 'highLW_L']
expBias = {'longwaveBias': {'lowLW_L': -25, 'lowLW_S': -10, 'highLW_S': 10, 'highLW_L': 25}}

expNames['shortwaveBias'] = ['lowSW_L','lowSW_S', 'highSW_S', 'highSW_L']
expBias['shortwaveBias'] = {'lowSW_L': -80, 'lowSW_S': -30, 'highSW_S': 30, 'highSW_L': 80}

expNames['specHumidBias'] = ['lowQS_L','lowQS_S', 'highQS_S', 'highQS_L']
expBias['specHumidBias'] = {'lowQS_L': -25,  'lowQS_S': -10, 'highQS_S': 10, 'highQS_L': 25}

expNames['airTempBias'] = ['lowTair_L','lowTair_S', 'highTair_S', 'highTair_L']
expBias['airTempBias'] = {'lowTair_L': -3, 'lowTair_S': -1.5, 'highTair_S': 1.5, 'highTair_L': 3}

expNames['windBias'] = ['lowU_L','lowU_S', 'highU_S', 'highU_L']
expBias['windBias'] = {'lowU_L': -3, 'lowU_S': -1.5, 'highU_S': 1.5, 'highU_L': 3}

expNames['Obs'] = 'Obs'
expBias['Obs'] = 0.

# ----------
# Scalar variables to extract from output
scalar_data_vars = ['scalarRainPlusMelt', 'scalarSWE', 'scalarSnowSublimation', 
                    'scalarInfiltration', 'scalarSurfaceRunoff', 'scalarSurfaceTemp',
                    'scalarSenHeatTotal', 'scalarLatHeatTotal', 'scalarSnowDepth']

# First look at model results
## Internal